# PSK II. Makró blokk beadandó

### Szabályok

  - Lehet (de nem kötelező) maximum három fós csoportokban dolgozni
  - A megoldásokat küldjétek el a martin.stancsics@gmail.com email címre a szakmai szabályzatban meghatározott határidőig

### Instrukciók

  - TODO

In [23]:
import dolo

## 1. feladat: Egy kalibrált RBC modell

Választott ország: #############

In [24]:
model_baseline = dolo.yaml_import("rbc_baseline.yaml")
print(model_baseline)


        Model:
        ------
        name: "None"
        type: "dtcc"
        file: "<string>

Equations:
----------

transition
 1  : 0.0000 : a[t] = exp((rho)*(log(a[t-1])) + eps[t])
 2  : 0.0000 : k[t] = (1 - (delta))*(k[t-1]) + i[t-1]

arbitrage
 1  : 0.1462 : (-(1))/(c[t]) + ((beta)*(1 + r[t+1] - (delta)))*((1)/(c[t+1]))
 2  : 5.2169 : -(w[t]) + ((phi)*(c[t]))/(1 - (l[t]))

definitions
 1  : y[t] = ((exp(a[t]))*((k[t])^(alpha)))*((l[t])^(1 - (alpha)))
 2  : c[t] = y[t] - (i[t])
 3  : r[t] = ((alpha)*(y[t]))/(k[t])
 4  : w[t] = ((1 - (alpha))*(y[t]))/(l[t])




### Steady state

In [25]:
ss_baseline = dolo.algos.steady_state.find_steady_state(model_baseline)
model_baseline.set_calibration(**ss_baseline.flat)

Ellenőrizd, hogy valóban a steady-statet találtad-e meg

In [26]:
print(model_baseline)


        Model:
        ------
        name: "None"
        type: "dtcc"
        file: "<string>

Equations:
----------

transition
 1  : 0.0000 : a[t] = exp((rho)*(log(a[t-1])) + eps[t])
 2  : 0.0000 : k[t] = (1 - (delta))*(k[t-1]) + i[t-1]

arbitrage
 1  : 0.0000 : (-(1))/(c[t]) + ((beta)*(1 + r[t+1] - (delta)))*((1)/(c[t+1]))
 2  : 0.0000 : -(w[t]) + ((phi)*(c[t]))/(1 - (l[t]))

definitions
 1  : y[t] = ((exp(a[t]))*((k[t])^(alpha)))*((l[t])^(1 - (alpha)))
 2  : c[t] = y[t] - (i[t])
 3  : r[t] = ((alpha)*(y[t]))/(k[t])
 4  : w[t] = ((1 - (alpha))*(y[t]))/(l[t])




In [27]:
model_baseline.get_calibration()

{'beta': 0.94,
 'phi': 4.0,
 'delta': 0.06,
 'alpha': 0.33,
 'rho': 0.65,
 'l': 0.16623609157181168,
 'a': 0.9999999999999999,
 'r': 0.12382978723404253,
 'k': 3.193614388771363,
 'i': 0.19161686332628192,
 'c': 0.2492415987079727,
 'w': 1.7768413993019831,
 'name': 0.0,
 'y': 1.1983775462701163,
 'eps': 0}

### Szimuláció

In [28]:
A modell megoldása:

dr_baseline = dolo.approximate_controls(model_baseline)

Szimulálj 1000 időszaknyi adatot

In [29]:
sim_baseline = dolo.simulate(model_baseline, dr_baseline, N=1, T=1000).squeeze()
sim_df_baseline = sim_baseline \
    .to_dataframe(name="value") \
    .drop(columns="N") \
    .reset_index() \
    .pivot(index="T", columns="V") \
    ["value"]
sim_df_baseline.head()

V,a,c,eps,i,k,l,r,w,y
T,,,,,,,,,
0,1.000000,1.006761,0.049671,0.191617,3.193614,0.166236,0.123830,4.829956,1.198378
1,1.050926,1.033922,-0.013826,0.270281,3.193614,0.174811,0.134765,4.998639,1.304203
2,1.018631,1.028292,0.064769,0.211649,3.272278,0.168091,0.125045,4.942332,1.239941
3,1.079791,1.068123,0.152303,0.304419,3.287591,0.178139,0.137772,5.162278,1.372542
4,1.224095,1.220299,-0.023415,0.515406,3.394754,0.200689,0.168726,5.794639,1.735705


### Kiértékelés

Átlaghoz viszonyított szórások:

In [30]:
sim_df_baseline.std() / sim_df_baseline.mean()

V
a       0.129257
c       0.155670
eps    50.652447
i       0.919462
k       0.199660
l       0.119561
r       0.218632
w       0.152410
y       0.242500
dtype: float64

Korrelációk:

In [31]:
sim_df_baseline.corr()

V,a,c,eps,i,k,l,r,w,y
V,,,,,,,,,
a,1.000000,0.786517,-0.008362,0.931837,0.434380,0.872505,0.671563,0.848630,0.986232
c,0.786517,1.000000,0.000609,0.530599,0.843809,0.413840,0.132341,0.981178,0.859677
eps,-0.008362,0.000609,1.000000,-0.010828,0.003903,-0.011384,-0.018999,-0.001650,-0.006197
i,0.931837,0.530599,-0.010828,1.000000,0.077916,0.990336,0.887957,0.601435,0.889141
k,0.434380,0.843809,0.003903,0.077916,1.000000,-0.061102,-0.359040,0.838628,0.502531
l,0.872505,0.413840,-0.011384,0.990336,-0.061102,1.000000,0.938947,0.485480,0.820279
r,0.671563,0.132341,-0.018999,0.887957,-0.359040,0.938947,1.000000,0.195622,0.606597
w,0.848630,0.981178,-0.001650,0.601435,0.838628,0.485480,0.195622,1.000000,0.892206
y,0.986232,0.859677,-0.006197,0.889141,0.502531,0.820279,0.606597,0.892206,1.000000


**Kérdések:**

  - Mennyire adja vissza jól a modell a megfigyelt korrelációkat?
  - Mi a helyzet a munka szórásával?
  - Mi a helyzet a munka prociklikusságával (munka-GDP korreláció)?
  - Mi lehet ez utóbbinak az oka?

## 2. feladat: Oszthatatlan munka

Ötlet: a háztartás nem döntheti el, hogy hány órát dolgozik, csak azt, hogy milyen valószínűséggel.
  - ü

In [32]:
model_alternative = dolo.yaml_import("rbc_indivisible_labor.yaml")
print(model_alternative)


        Model:
        ------
        name: "None"
        type: "dtcc"
        file: "<string>

Equations:
----------

transition
 1  : 0.0000 : a[t] = exp((rho)*(log(a[t-1])) + eps[t])
 2  : 0.0000 : k[t] = (1 - (delta))*(k[t-1]) + i[t-1]

arbitrage
 1  : 0.1462 : (-(1))/(c[t]) + ((beta)*(1 + r[t+1] - (delta)))*((1)/(c[t+1]))
 2  : 5.2169 : -(w[t]) + ((phi)*(c[t]))/(1 - (l[t]))

definitions
 1  : y[t] = ((exp(a[t]))*((k[t])^(alpha)))*((l[t])^(1 - (alpha)))
 2  : c[t] = y[t] - (i[t])
 3  : r[t] = ((alpha)*(y[t]))/(k[t])
 4  : w[t] = ((1 - (alpha))*(y[t]))/(l[t])




### Steady state

In [25]:
ss_alternative = dolo.algos.steady_state.find_steady_state(model_alternative)
model_alternative.set_calibration(**ss_alternative.flat)

Ellenőrizd, hogy valóban a steady-statet találtad-e meg

In [26]:
print(model_alternative)


        Model:
        ------
        name: "None"
        type: "dtcc"
        file: "<string>

Equations:
----------

transition
 1  : 0.0000 : a[t] = exp((rho)*(log(a[t-1])) + eps[t])
 2  : 0.0000 : k[t] = (1 - (delta))*(k[t-1]) + i[t-1]

arbitrage
 1  : 0.0000 : (-(1))/(c[t]) + ((beta)*(1 + r[t+1] - (delta)))*((1)/(c[t+1]))
 2  : 0.0000 : -(w[t]) + ((phi)*(c[t]))/(1 - (l[t]))

definitions
 1  : y[t] = ((exp(a[t]))*((k[t])^(alpha)))*((l[t])^(1 - (alpha)))
 2  : c[t] = y[t] - (i[t])
 3  : r[t] = ((alpha)*(y[t]))/(k[t])
 4  : w[t] = ((1 - (alpha))*(y[t]))/(l[t])




In [27]:
model_alternative.get_calibration()

{'beta': 0.94,
 'phi': 4.0,
 'delta': 0.06,
 'alpha': 0.33,
 'rho': 0.65,
 'l': 0.16623609157181168,
 'a': 0.9999999999999999,
 'r': 0.12382978723404253,
 'k': 3.193614388771363,
 'i': 0.19161686332628192,
 'c': 0.2492415987079727,
 'w': 1.7768413993019831,
 'name': 0.0,
 'y': 1.1983775462701163,
 'eps': 0}

### Szimuláció

A modell megoldása:

In [ ]:
dr_alternative = dolo.approximate_controls(model_alternative)

Szimulálj 1000 időszaknyi adatot

In [29]:
sim_alternative = dolo.simulate(model_alternative, dr_alternative, N=1, T=1000).squeeze()
sim_df_alternative = sim_alternative \
    .to_dataframe(name="value") \
    .drop(columns="N") \
    .reset_index() \
    .pivot(index="T", columns="V") \
    ["value"]
sim_df_alternative.head()

V,a,c,eps,i,k,l,r,w,y
T,,,,,,,,,
0,1.000000,1.006761,0.049671,0.191617,3.193614,0.166236,0.123830,4.829956,1.198378
1,1.050926,1.033922,-0.013826,0.270281,3.193614,0.174811,0.134765,4.998639,1.304203
2,1.018631,1.028292,0.064769,0.211649,3.272278,0.168091,0.125045,4.942332,1.239941
3,1.079791,1.068123,0.152303,0.304419,3.287591,0.178139,0.137772,5.162278,1.372542
4,1.224095,1.220299,-0.023415,0.515406,3.394754,0.200689,0.168726,5.794639,1.735705


### Kiértékelés

Átlaghoz viszonyított szórások:

In [30]:
sim_df_alternative.std() / sim_df_alternative.mean()

V
a       0.129257
c       0.155670
eps    50.652447
i       0.919462
k       0.199660
l       0.119561
r       0.218632
w       0.152410
y       0.242500
dtype: float64

Korrelációk:

In [31]:
sim_df_alternative.corr()

V,a,c,eps,i,k,l,r,w,y
V,,,,,,,,,
a,1.000000,0.786517,-0.008362,0.931837,0.434380,0.872505,0.671563,0.848630,0.986232
c,0.786517,1.000000,0.000609,0.530599,0.843809,0.413840,0.132341,0.981178,0.859677
eps,-0.008362,0.000609,1.000000,-0.010828,0.003903,-0.011384,-0.018999,-0.001650,-0.006197
i,0.931837,0.530599,-0.010828,1.000000,0.077916,0.990336,0.887957,0.601435,0.889141
k,0.434380,0.843809,0.003903,0.077916,1.000000,-0.061102,-0.359040,0.838628,0.502531
l,0.872505,0.413840,-0.011384,0.990336,-0.061102,1.000000,0.938947,0.485480,0.820279
r,0.671563,0.132341,-0.018999,0.887957,-0.359040,0.938947,1.000000,0.195622,0.606597
w,0.848630,0.981178,-0.001650,0.601435,0.838628,0.485480,0.195622,1.000000,0.892206
y,0.986232,0.859677,-0.006197,0.889141,0.502531,0.820279,0.606597,0.892206,1.000000


**Kérdések:**

  - Mi változott?
  - Mi a helyzet a munka szórásával?
  - Mi a helyzet a munka prociklikusságával (munka-GDP korreláció)?